## LLM Validation walkthrough

Here we demonstrate a sample process of running the LLM model validation. The steps are as follows:
1. Give task specifications
2. Download dataset + prompt + model
3. Data ETL to align input/output formats
4. Define relevant components (customize if necessary)
5. Execute the orchestration process
6. Analyze the results